In [1]:
mllib = r'C:/Users/zlwan/AppData/Roaming/Python/Python37/site-packages/pyspark/data/mllib'

from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [7]:
training = spark.read.format('libsvm').load(f'{mllib}/sample_libsvm_data.txt')
training.show(1)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
+-----+--------------------+
only showing top 1 row



In [15]:
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')
lrModel = lr.fit(training)

In [ ]:
lrModel.coefficients

In [ ]:

lrModel.intercept

In [28]:
training_sum = lrModel.summary

In [29]:
training_sum.r2

0.9999998176861554

In [30]:
training_sum.rootMeanSquaredError

0.00021138856000615784

In [33]:
all_data = spark.read.format('libsvm').load(f'{mllib}/sample_libsvm_data.txt')

In [38]:
split_data = all_data.randomSplit([0.7,0.3])
split_data

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [43]:
train_data, test_data = split_data
train_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|                72|
|   mean|0.6111111111111112|
| stddev|0.4909191022411197|
|    min|               0.0|
|    max|               1.0|
+-------+------------------+



In [44]:
test_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|                28|
|   mean|0.4642857142857143|
| stddev|  0.50787450018337|
|    min|               0.0|
|    max|               1.0|
+-------+------------------+



In [49]:
correct_model = lr.fit(train_data)
test_result = correct_model.evaluate(test_data)
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.1281622815860738|
|  -0.622047383022618|
|  1.8609295328355555|
| -1.0359261429688003|
|-0.11019489336397736|
|  -5.794379909171163|
|0.044511660630623506|
| 0.20866157541281471|
|9.157468815613745E-4|
|-0.09513332184829326|
|-0.08401187239620189|
|  0.3931289811445575|
| 0.12099873242751713|
| 0.00642974985669631|
|-0.18391525641468998|
|-0.02124293822689...|
|-0.00311512957630...|
|-0.02206325816814...|
|0.012326489513020733|
|-0.02740935364624...|
+--------------------+
only showing top 20 rows



In [50]:
test_result.rootMeanSquaredError

1.1772400270736456

In [51]:
unlabeled_data = test_data.select('features')

In [52]:
predictions = correct_model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(692,[98,99,100,1...|  0.1281622815860738|
|(692,[100,101,102...|   0.622047383022618|
|(692,[124,125,126...| -1.8609295328355555|
|(692,[124,125,126...|  1.0359261429688003|
|(692,[124,125,126...| 0.11019489336397736|
|(692,[125,126,127...|   5.794379909171163|
|(692,[126,127,128...|-0.04451166063062...|
|(692,[126,127,128...|-0.20866157541281471|
|(692,[126,127,128...|-9.15746881561374...|
|(692,[128,129,130...| 0.09513332184829326|
|(692,[150,151,152...| 0.08401187239620189|
|(692,[152,153,154...| -0.3931289811445575|
|(692,[153,154,155...|-0.12099873242751713|
|(692,[154,155,156...|-0.00642974985669631|
|(692,[154,155,156...| 0.18391525641468998|
|(692,[124,125,126...|   1.021242938226895|
|(692,[126,127,128...|  1.0031151295763014|
|(692,[126,127,128...|  1.0220632581681492|
|(692,[127,128,155...|  0.9876735104869793|
|(692,[128,129,130...|   1.02740